# Documentation

[Python Github Repo](https://github.com/CFBD/cfbd-python/tree/main)


# Install CFBD and perform Imports

In [ ]:
pip install -U cfbd

In [ ]:
import requests, pandas as pd
from getpass import getpass
import numpy as np
import cfbd
import time
import cfbd
from cfbd.models.team import Team
from cfbd.rest import ApiException
from pprint import pprint

from cfbd.models.division_classification import DivisionClassification
from cfbd.models.game import Game
from cfbd.models.season_type import SeasonType

from cfbd.models.coach import Coach

from collections import defaultdict

from dataclasses import dataclass, asdict

from datetime import datetime, timedelta, timezone
from pydantic import BaseModel, Field, StrictBool, StrictFloat, StrictInt, StrictStr

from cfbd import GameLine, BettingGame, GameWeather

# Constants

In [ ]:
# Update This every week to pull the latest data
CURR_WEEK = 4

# Class Definitions

In [631]:
@dataclass
class GameRecord:
  id: int
  season: int # Game
  week: int # Game
  start_date: datetime # Game
  neutral: int # Game
  conference_game: int # Game
  venue: int # Game
  home_team: int # Game
  home_conference: str # Game
  home_classification: str # Game
  home_pregame_elo: int # Game
  away_team: int # Game
  away_conference: str # Game
  away_classification: str # Game
  away_pregame_elo: int # Game
  home_coach_name: str # Coach
  home_coach_tenure: int # Coach, derived
  home_coach_interim: int # Coach, derived
  away_coach_name: str # Coach
  away_coach_tenure: int # Coach, derived
  away_coach_interim: int # Coach, derived
  spread: float # GameLine
  spread_open: float # GameLine
  over_under: float # GameLine
  over_under_open: float # GameLine
  home_moneyline: float # GameLine
  away_moneyline: float # GameLine
  game_indoors: int # GameWeather
  temperature: float # GameWeather
  dew_point: float # GameWeather
  humidity: float # GameWeather
  precipitation: float # GameWeather
  snowfall: float # GameWeather
  wind_dir: float # GameWeather
  wind_speed: float # GameWeather
  atm_pressure: float # GameWeather
  weather_condition: int # GameWeather
  # result: int # Game, derived
  covered: bool




  def __init__(self, id=None, season=None, week=None, start_date=None, neutral=0,
               conference_game=0, venue="", home_team=-1, home_conference="",
               home_classification="", home_pregame_elo=-1, away_team=-1,
               away_conference="", away_classification="", away_pregame_elo=-1,
               home_coach_name="", home_coach_tenure=-1,
               home_coach_interim=0, away_coach_name = "",
               away_coach_tenure=-1, away_coach_interim=0,
               spread=float("inf"), spread_open=float("inf"), over_under=-1,
               over_under_open=-1, home_moneyline=0, away_moneyline=0,
               game_indoors=0, temperature=float("inf"),
               dew_point=float("inf"), humidity=-1, precipitation=-1,
               snowfall=-1, wind_dir=float("inf"),wind_speed=-1,
               pressure=float("inf"), weather_condition=-1, covered=None):
    self.id = id
    self.season=season
    self.week = week
    self.start_date = start_date
    self.neutral = neutral
    self.conference_game = conference_game
    self.venue = venue
    self.home_team = home_team
    self.home_conference = home_conference
    self.home_classification = home_classification
    self.home_pregame_elo = home_pregame_elo
    self.away_team = away_team
    self.away_conference = away_conference
    self.away_classification = away_classification
    self.away_pregame_elo = away_pregame_elo
    self.home_coach_name = home_coach_name
    self.home_coach_tenure = home_coach_tenure
    self.home_coach_interim = home_coach_interim
    self.away_coach_name = away_coach_name
    self.away_coach_tenure = away_coach_tenure
    self.away_coach_interim = away_coach_interim
    self.spread = spread
    self.spread_open = spread_open
    self.over_under = over_under
    self.over_under_open = over_under_open
    self.home_moneyline = home_moneyline
    self.away_moneyline = away_moneyline
    self.game_indoors = game_indoors
    self.temperature = temperature
    self.dew_point = dew_point
    self.humidity = humidity
    self.precipitation = precipitation
    self.snowfall = snowfall
    self.wind_dir = wind_dir
    self.wind_speed = wind_speed
    self.atm_pressure = pressure
    self.weather_condition = weather_condition
    self.covered = covered
    return

  def to_dict(self):
    return asdict(self)

# Utility

In [ ]:
# Free API key available at collegefootballdata.com
API_KEY = getpass("Enter API Key")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

Enter API Key··········


In [ ]:
configuration = cfbd.Configuration(
    access_token = API_KEY
)

In [ ]:
teams_dict = defaultdict(dict)
games_dict = defaultdict(dict)
coaches_dict = defaultdict(list)
weather_dict = defaultdict(dict)
lines_dict = defaultdict(GameLine)

# Retrieve Data

**Get 2025 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2025

    try:
        games_2025 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2025[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401756846, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2025, 8, 23, 16, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=True, attendance=None, venue_id=3504, venue='Aviva Stadium', home_id=2306, home_team='Kansas State', home_conference='Big 12', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=21, home_line_scores=[0, 7, 0, 14], home_postgame_win_probability=0.20154441893100739, home_pregame_elo=1669, home_postgame_elo=1666, away_id=66, away_team='Iowa State', away_conference='Big 12', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=24, away_line_scores=[7, 0, 7, 10], away_postgame_win_probability=0.7984555810689926, away_pregame_elo=1616, away_postgame_elo=1619, excitement_index=6.029094160739601, highlights='', notes='Aer Lingus College Football Classic'),
 Game(id=401767476, season=2025,

**Get 2025 Subset**

In [627]:
games_2025[0].to_dict()

{'id': 401756846,
 'season': 2025,
 'week': 1,
 'seasonType': <SeasonType.REGULAR: 'regular'>,
 'startDate': datetime.datetime(2025, 8, 23, 16, 0, tzinfo=datetime.timezone.utc),
 'startTimeTBD': False,
 'completed': True,
 'neutralSite': True,
 'conferenceGame': True,
 'venueId': 3504,
 'venue': 'Aviva Stadium',
 'homeId': 2306,
 'homeTeam': 'Kansas State',
 'homeConference': 'Big 12',
 'homeClassification': <DivisionClassification.FBS: 'fbs'>,
 'homePoints': 21,
 'homeLineScores': [0, 7, 0, 14],
 'homePostgameWinProbability': 0.20154441893100739,
 'homePregameElo': 1669,
 'homePostgameElo': 1666,
 'awayId': 66,
 'awayTeam': 'Iowa State',
 'awayConference': 'Big 12',
 'awayClassification': <DivisionClassification.FBS: 'fbs'>,
 'awayPoints': 24,
 'awayLineScores': [7, 0, 7, 10],
 'awayPostgameWinProbability': 0.7984555810689926,
 'awayPregameElo': 1616,
 'awayPostgameElo': 1619,
 'excitementIndex': 6.029094160739601,
 'highlights': '',
 'notes': 'Aer Lingus College Football Classic',
 '

In [ ]:
def get_curr_year_subset(games: list[Game], from_week: int):
  return [game for game in games if game.week <= CURR_WEEK and game.week >= from_week]

In [ ]:
games_2025_subset = get_curr_year_subset(games_2025, 0)

#Sanity check
# max([game.week for game in games_2025_subset])
# returns 4

In [664]:
home_teams = set({"Rutgers", "Clemson", "Wisconsin", "TCU", "Utah", "Nebraska",
                  "Oklahoma", "UCF", "Duke", "Kansas", "Miami", "Indiana",
                  "Washington State", "Baylor", "USC"})
bet_games = [game for game in games_2025_subset if game.home_team in home_teams and game.week == CURR_WEEK]
for game in bet_games:
  # print(game.home_team, game.away_team, game.id)
  print(game.id)
  print(game.home_team)
  print(game.away_team)
  print()

401752844
Rutgers
Iowa

401756901
Utah
Texas Tech

401754636
TCU
SMU

401752845
Wisconsin
Maryland

401754537
Clemson
Syracuse

401752846
Nebraska
Michigan

401752714
Oklahoma
Auburn

401754640
UCF
North Carolina

401754538
Duke
NC State

401756899
Kansas
West Virginia

401756896
Baylor
Arizona State

401752709
Miami
Florida

401752842
Washington State
Washington

401752843
Indiana
Illinois

401752847
USC
Michigan State



In [ ]:
bt_2025 = [game for game in games_2025 if game.home_conference == "Big Ten"]
bt_2025

[Game(id=401752794, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2025, 8, 28, 22, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=False, conference_game=False, attendance=None, venue_id=6577, venue='SHI Stadium', home_id=164, home_team='Rutgers', home_conference='Big Ten', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=34, home_line_scores=[7, 24, 0, 3], home_postgame_win_probability=0.9000661373138428, home_pregame_elo=1522, home_postgame_elo=1524, away_id=195, away_team='Ohio', away_conference='Mid-American', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=31, away_line_scores=[7, 10, 14, 0], away_postgame_win_probability=0.09993386268615723, away_pregame_elo=1621, away_postgame_elo=1619, excitement_index=6.943864856309906, highlights='', notes=None),
 Game(id=401752793, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.

**Get 2024 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2024

    try:
        games_2024 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2024[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# I fixed the code, so this won't be needed
games_2024 = games_response[:]

**Get 2023 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2023

    try:
        games_2023 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2023[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401525434, season=2023, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2023, 8, 26, 18, 30, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=False, attendance=49000, venue_id=3504, venue='Aviva Stadium', home_id=87, home_team='Notre Dame', home_conference='FBS Independents', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=42, home_line_scores=[14, 14, 7, 7], home_postgame_win_probability=0.998958343504533, home_pregame_elo=1733, home_postgame_elo=1819, away_id=2426, away_team='Navy', away_conference='American Athletic', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=3, away_line_scores=[0, 0, 0, 3], away_postgame_win_probability=0.0010416564954669472, away_pregame_elo=1471, away_postgame_elo=1385, excitement_index=1.3469076611, highlights='', notes=None),
 Game(id=401540199, season=2023, week=1, season_type=<

**2022 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2022

    try:
        games_2022 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2022[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401426532, season=2022, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2022, 8, 27, 16, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=False, conference_game=False, attendance=13688, venue_id=3796, venue='Houchens Industries-L.T. Smith Stadium', home_id=98, home_team='Western Kentucky', home_conference='Conference USA', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=38, home_line_scores=[7, 14, 0, 17], home_postgame_win_probability=0.9542764266226265, home_pregame_elo=1684, home_postgame_elo=1684, away_id=2046, away_team='Austin Peay', away_conference='Atlantic Sun', away_classification=<DivisionClassification.FCS: 'fcs'>, away_points=27, away_line_scores=[10, 7, 3, 7], away_postgame_win_probability=0.04572357337737354, away_pregame_elo=None, away_postgame_elo=None, excitement_index=2.6135509766, highlights='', notes=None),
 Game(id=401405059, se

**2021 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2021

    try:
        games_2021 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2021[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401282714, season=2021, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2021, 8, 28, 17, 20, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=False, conference_game=True, attendance=41064, venue_id=3832, venue='Memorial Stadium (Champaign, IL)', home_id=356, home_team='Illinois', home_conference='Big Ten', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=30, home_line_scores=[2, 14, 14, 0], home_postgame_win_probability=0.4056606386617034, home_pregame_elo=1392, home_postgame_elo=1411, away_id=158, away_team='Nebraska', away_conference='Big Ten', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=22, away_line_scores=[0, 9, 7, 6], away_postgame_win_probability=0.5943393613382966, away_pregame_elo=1503, away_postgame_elo=1484, excitement_index=5.4859812562, highlights='', notes=None),
 Game(id=401286187, season=2021, week=1, season_type=

**2020 Games**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.GamesApi(api_client)
    year = 2020

    try:
        games_2020 = api_instance.get_games(year=year, season_type="regular")
        print("The response of GamesApi->get_games:\n")
        pprint(games_2020[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_games: %s\n" % e)

The response of GamesApi->get_games:

[Game(id=401218616, season=2020, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2020, 8, 30, 1, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=False, attendance=None, venue_id=3689, venue='Cramton Bowl', home_id=2110, home_team='Central Arkansas', home_conference='Southland', home_classification=<DivisionClassification.FCS: 'fcs'>, home_points=24, home_line_scores=[0, 6, 7, 11], home_postgame_win_probability=None, home_pregame_elo=None, home_postgame_elo=None, away_id=2046, away_team='Austin Peay', away_conference='OVC', away_classification=<DivisionClassification.FCS: 'fcs'>, away_points=17, away_line_scores=[7, 3, 0, 7], away_postgame_win_probability=None, away_pregame_elo=None, away_postgame_elo=None, excitement_index=None, highlights='', notes=None),
 Game(id=401238035, season=2020, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datet

**Coaches**

**Get 2025 Coaches**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2025

    try:
        coaches_2025 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2025[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Scott', last_name='Abell', hire_date=None, seasons=[CoachSeason(school='Rice', year=2025, games=0, wins=0, losses=0, ties=0, preseason_rank=None, postseason_rank=None, srs=-12.3, sp_overall=-11.2, sp_offense=17.7, sp_defense=28.7)]),
 Coach(first_name='Tim', last_name='Albin', hire_date=None, seasons=[CoachSeason(school='Charlotte', year=2025, games=0, wins=0, losses=0, ties=0, preseason_rank=None, postseason_rank=None, srs=-34.3, sp_overall=-17.3, sp_offense=19, sp_defense=36.3)]),
 Coach(first_name='Major', last_name='Applewhite', hire_date=datetime.datetime(2024, 1, 18, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='South Alabama', year=2025, games=0, wins=0, losses=0, ties=0, preseason_rank=None, postseason_rank=None, srs=-5.9, sp_overall=-3.3, sp_offense=27.3, sp_defense=30.6)]),
 Coach(first_name='Dave', last_name='Aranda', hire_date=datetime.datetime(2020, 1, 22, 0, 0, tzinfo=datetime.timezone.utc), 

**Get 2024 Coaches**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2024

    try:
        coaches_2024 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2024[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Pete', last_name='Alamar', hire_date=None, seasons=[CoachSeason(school='Rice', year=2024, games=4, wins=2, losses=2, ties=0, preseason_rank=None, postseason_rank=None, srs=-8.6, sp_overall=-11.4, sp_offense=20.5, sp_defense=30.4)]),
 Coach(first_name='Tim', last_name='Albin', hire_date=datetime.datetime(2021, 7, 14, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Ohio', year=2024, games=14, wins=11, losses=3, ties=0, preseason_rank=None, postseason_rank=None, srs=1.7, sp_overall=2.3, sp_offense=27.2, sp_defense=23.9)]),
 Coach(first_name='Blake', last_name='Anderson', hire_date=datetime.datetime(2020, 12, 12, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Utah State', year=2024, games=0, wins=0, losses=0, ties=0, preseason_rank=None, postseason_rank=None, srs=-11.7, sp_overall=-10.4, sp_offense=30.2, sp_defense=40.3)]),
 Coach(first_name='Major', last_name='Applewhite', hire_date=datetime.

In [ ]:
# I fixed the retrieval, so this shouldn't be needed next time
coaches_2024 = coaches[:]

**2023 Coaches**

In [ ]:

with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2023

    try:
        coaches_2023 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2023[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Tim', last_name='Albin', hire_date=datetime.datetime(2021, 7, 14, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Ohio', year=2023, games=13, wins=10, losses=3, ties=0, preseason_rank=None, postseason_rank=None, srs=-5.8, sp_overall=-1.6, sp_offense=18.5, sp_defense=18.1)]),
 Coach(first_name='Tom', last_name='Allen', hire_date=datetime.datetime(2016, 12, 1, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Indiana', year=2023, games=12, wins=3, losses=9, ties=0, preseason_rank=None, postseason_rank=None, srs=-3.5, sp_overall=-7.4, sp_offense=19.6, sp_defense=27.2)]),
 Coach(first_name='Blake', last_name='Anderson', hire_date=datetime.datetime(2020, 12, 12, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Utah State', year=2023, games=13, wins=6, losses=7, ties=0, preseason_rank=None, postseason_rank=None, srs=-4.6, sp_overall=-10.9, sp_offense=28.6, sp_defense=38.6)]),
 Coac

**2022 Coaches**


In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2022

    try:
        coaches_2022 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2022[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Shaun', last_name='Aguano', hire_date=datetime.datetime(2022, 9, 18, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Arizona State', year=2022, games=9, wins=2, losses=7, ties=0, preseason_rank=None, postseason_rank=None, srs=-3.9, sp_overall=-4.1, sp_offense=30.5, sp_defense=35.7)]),
 Coach(first_name='Tim', last_name='Albin', hire_date=datetime.datetime(2021, 7, 14, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Ohio', year=2022, games=14, wins=10, losses=4, ties=0, preseason_rank=None, postseason_rank=None, srs=-3.6, sp_overall=-4.6, sp_offense=31.6, sp_defense=35.2)]),
 Coach(first_name='Josh', last_name='Aldridge', hire_date=datetime.datetime(2022, 11, 29, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Liberty', year=2022, games=1, wins=0, losses=1, ties=0, preseason_rank=None, postseason_rank=None, srs=-3.5, sp_overall=-1.5, sp_offense=24.1, sp_defense=25.4)]),
 Co

**2021 Coaches**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2021

    try:
        coaches_2021 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2021[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Steve', last_name='Addazio', hire_date=datetime.datetime(2019, 12, 10, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Colorado State', year=2021, games=12, wins=3, losses=9, ties=0, preseason_rank=None, postseason_rank=None, srs=-0.5, sp_overall=-2.4, sp_offense=23.3, sp_defense=25.9)]),
 Coach(first_name='Tim', last_name='Albin', hire_date=datetime.datetime(2021, 7, 14, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Ohio', year=2021, games=12, wins=3, losses=9, ties=0, preseason_rank=None, postseason_rank=None, srs=-13.8, sp_overall=-11.6, sp_offense=25.1, sp_defense=35.2)]),
 Coach(first_name='Tom', last_name='Allen', hire_date=datetime.datetime(2016, 12, 1, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Indiana', year=2021, games=12, wins=2, losses=10, ties=0, preseason_rank=17, postseason_rank=None, srs=-6.9, sp_overall=-7.8, sp_offense=20.4, sp_defense=27.3)]),
 Co

**2020 Coaches**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.CoachesApi(api_client)
    year = 2020

    try:
        coaches_2020 = api_instance.get_coaches(year=year)
        print("The response of CoachesApi->get_coaches:\n")
        pprint(coaches_2020[:5])
    except Exception as e:
        print("Exception when calling CoachesApi->get_coaches: %s\n" % e)

The response of CoachesApi->get_coaches:

[Coach(first_name='Steve', last_name='Addazio', hire_date=datetime.datetime(2019, 12, 10, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Colorado State', year=2020, games=4, wins=1, losses=3, ties=0, preseason_rank=None, postseason_rank=None, srs=-11.3, sp_overall=-6.5, sp_offense=24.2, sp_defense=30.5)]),
 Coach(first_name='Tom', last_name='Allen', hire_date=datetime.datetime(2016, 12, 1, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Indiana', year=2020, games=8, wins=6, losses=2, ties=0, preseason_rank=None, postseason_rank=12, srs=13.2, sp_overall=11.2, sp_offense=33.6, sp_defense=22.6)]),
 Coach(first_name='Gary', last_name='Andersen', hire_date=datetime.datetime(2018, 12, 9, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Utah State', year=2020, games=3, wins=0, losses=3, ties=0, preseason_rank=None, postseason_rank=None, srs=-19.2, sp_overall=-16.7, sp_offense=20.2, sp_defense=37)])

**Coaches Class**

In [ ]:
@dataclass
class CoachLite:
  name: str
  tenure: int
  fired: bool
  interim: bool
  last_week_coached: int
  season: int
  season_games_coached: int

  def __init__(self, name="", tenure=0, fired=False, interim=False,
               last_week_coached=None, season=None, season_games_coached=-1):
    self.name = name
    self.tenure = tenure
    self.fired = fired
    self.interim=interim
    self.last_week_coached=last_week_coached
    self.season = season
    self.season_games_coached=season_games_coached

  def to_dict(self):
    return asdict(self)

**Build Coaches Map**

In [ ]:
def update_coaches_dict(coaches: list[Coach], curr_year: int, games_response: list[Game]):

  def calculate_tenure(hire_date: datetime):
    # Calculate tenure to be the number of full seasons a coach has been HC
    if interim:
      return 0
    reduce_tenure = hire_date.month > 8
    return curr_year - (hire_date.year + reduce_tenure)

  def calculate_coaching_change():
    try:
      existing_coach = coaches_dict[(school, curr_year)][0]
      # To handle a weird case in 2021 data with Utah St
      if coach_to_add.name == existing_coach.name:
        return
      if coach_to_add.interim:
        interim = coach_to_add
        fired = existing_coach
      else:
        interim = existing_coach
        fired = coach_to_add
      fired.fired = True
      team_games = sorted([game.week for game in games_response if game.home_team == school or game.away_team == school])
      # Set the last week the fired coach was HC for proper game record data
      fired.last_week_coached = team_games[fired.season_games_coached - 1]
      coaches_dict[(school, curr_year)] = [fired, interim]
    except:
      print(f"Exception occurred in {school}, {curr_year}")
    finally:
      return

  for coach in coaches:
    season = coach.seasons[0]
    school = season.school
    name = " ".join([coach.first_name, coach.last_name])
    interim = coach.hire_date is None
    tenure = calculate_tenure(coach.hire_date)
    games_coached = season.games
    coach_to_add = CoachLite(name = name, season = curr_year, interim=interim,
                             tenure=tenure, season_games_coached=games_coached)
    if (school, curr_year) in coaches_dict:
      calculate_coaching_change()
    else:
      coaches_dict[(school, curr_year)].append(coach_to_add)

In [ ]:
coaches_dict = defaultdict(list)
coaches = [coaches_2025, coaches_2024, coaches_2023, coaches_2022, coaches_2021, coaches_2020]
years = [2025, 2024, 2023, 2022, 2021, 2020]
games = [games_2025_subset, games_2024, games_2023, games_2022, games_2021, games_2020]

update_coaches_dict(coaches[0], years[0], games[0])
update_coaches_dict(coaches[1], years[1], games[1])
update_coaches_dict(coaches[2], years[2], games[2])
update_coaches_dict(coaches[3], years[3], games[3])
update_coaches_dict(coaches[4], years[4], games[4])

[CoachLite(name='Nick Saban', tenure=15, fired=False, interim=False, last_week_coached=None, season=2022, season_games_coached=13)]

**Get Lines 2025**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    season_type = "regular"
    year = 2025

    try:
        lines_2025 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2025[:5])
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401752842, season=2025, season_type=<SeasonType.REGULAR: 'regular'>, week=4, start_date=datetime.datetime(2025, 9, 20, 23, 30, tzinfo=datetime.timezone.utc), home_team_id=265, home_team='Washington State', home_conference='Pac-12', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=None, away_team_id=264, away_team='Washington', away_conference='Big Ten', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=None, lines=[GameLine(provider='DraftKings', spread=21, formatted_spread='Washington -21', spread_open=12.5, over_under=51.5, over_under_open=None, home_moneyline=900, away_moneyline=-1600), GameLine(provider='Bovada', spread=21, formatted_spread='Washington -21.0', spread_open=20.5, over_under=52.5, over_under_open=53.5, home_moneyline=750, away_moneyline=-1400), GameLine(provider='ESPN Bet', spread=21.5, formatted_spread='Washington -21.5', spread_open=20.5, over_under=52.5, over_under_open=53.

**Get Spreads 2024**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    season_type = "regular"
    year = 2024

    try:
        lines_2024 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2024[:5])
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401635575, season=2024, season_type=<SeasonType.REGULAR: 'regular'>, week=7, start_date=datetime.datetime(2024, 10, 12, 19, 30, tzinfo=datetime.timezone.utc), home_team_id=221, home_team='Pittsburgh', home_conference='ACC', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=17, away_team_id=25, away_team='California', away_conference='ACC', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=15, lines=[GameLine(provider='DraftKings', spread=-3.5, formatted_spread='Pittsburgh -3.5', spread_open=-2.5, over_under=57.5, over_under_open=None, home_moneyline=-166, away_moneyline=140), GameLine(provider='Bovada', spread=-3.5, formatted_spread='Pittsburgh -3.5', spread_open=-4.5, over_under=57.5, over_under_open=59, home_moneyline=-170, away_moneyline=145), GameLine(provider='ESPN Bet', spread=-3.5, formatted_spread='Pittsburgh -3.5', spread_open=-3.5, over_under=56.5, over_under_open=59.5, home_moneyline=

**Get Spreads 2023**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    year = 2023
    season_type = "regular"

    try:
        lines_2023 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2023)
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401520400, season=2023, season_type=<SeasonType.REGULAR: 'regular'>, week=10, start_date=datetime.datetime(2023, 11, 4, 19, 30, tzinfo=datetime.timezone.utc), home_team_id=113, home_team='Massachusetts', home_conference='FBS Independents', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=31, away_team_id=2771, away_team='Merrimack', away_conference='NEC', away_classification=<DivisionClassification.FCS: 'fcs'>, away_score=21, lines=[GameLine(provider='ESPN Bet', spread=-16.5, formatted_spread='Massachusetts -16.5', spread_open=None, over_under=57.5, over_under_open=None, home_moneyline=None, away_moneyline=None)]),
 BettingGame(id=401525451, season=2023, season_type=<SeasonType.REGULAR: 'regular'>, week=10, start_date=datetime.datetime(2023, 11, 4, 17, 0, tzinfo=datetime.timezone.utc), home_team_id=2534, home_team='Sam Houston', home_conference='Conference USA', home_classification=<DivisionClassification.FBS: '

**Get Lines 2022**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    year = 2022
    season_type = "regular"
    try:
        lines_2022 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2022[:5])
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401411101, season=2022, season_type=<SeasonType.REGULAR: 'regular'>, week=1, start_date=datetime.datetime(2022, 9, 6, 0, 0, tzinfo=datetime.timezone.utc), home_team_id=59, home_team='Georgia Tech', home_conference='ACC', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=10, away_team_id=228, away_team='Clemson', away_conference='ACC', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=41, lines=[GameLine(provider='teamrankings', spread=24.5, formatted_spread='Clemson -24.5', spread_open=None, over_under=51, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='consensus', spread=24.5, formatted_spread='Clemson -24.5', spread_open=None, over_under=51.5, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='William Hill (New Jersey)', spread=24, formatted_spread='Clemson -24', spread_open=None, over_under=51.5, over_under_open=None, home_m

**Get Lines 2021**


In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    year = 2021
    season_type = "regular"

    try:
        lines_2021 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2021[:5])
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401310752, season=2021, season_type=<SeasonType.REGULAR: 'regular'>, week=13, start_date=datetime.datetime(2021, 11, 26, 20, 30, tzinfo=datetime.timezone.utc), home_team_id=2005, home_team='Air Force', home_conference='Mountain West', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=48, away_team_id=2439, away_team='UNLV', away_conference='Mountain West', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=14, lines=[GameLine(provider='teamrankings', spread=-18.5, formatted_spread='Air Force -18.5', spread_open=None, over_under=49.5, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='consensus', spread=-18.5, formatted_spread='Air Force -18.5', spread_open=None, over_under=49.5, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='William Hill (New Jersey)', spread=-18.5, formatted_spread='Air Force -18.5', spread_open=None, over_und

**Get Lines 2020**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    api_instance = cfbd.BettingApi(api_client)
    year = 2020
    season_type = "regular"

    try:
        lines_2020 = api_instance.get_lines(year=year, season_type=season_type)
        print("The response of BettingApi->get_lines:\n")
        pprint(lines_2020[:5])
    except Exception as e:
        print("Exception when calling BettingApi->get_lines: %s\n" % e)

The response of BettingApi->get_lines:

[BettingGame(id=401237129, season=2020, season_type=<SeasonType.REGULAR: 'regular'>, week=11, start_date=datetime.datetime(2020, 11, 15, 0, 0, tzinfo=datetime.timezone.utc), home_team_id=57, home_team='Florida', home_conference='SEC', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=63, away_team_id=8, away_team='Arkansas', away_conference='SEC', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=35, lines=[GameLine(provider='teamrankings', spread=-17, formatted_spread='Florida -17', spread_open=None, over_under=61.5, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='numberfire', spread=-16.5, formatted_spread='Florida -16.5', spread_open=None, over_under=61.5, over_under_open=None, home_moneyline=None, away_moneyline=None), GameLine(provider='consensus', spread=-17, formatted_spread='Florida -17', spread_open=None, over_under=62, over_under_open=None, home_moneyline=None, away

**Build Lines Map**

In [ ]:
lines_dict = defaultdict(GameLine)
LINE_PREFERENCE = ["consensus", "Bovada"]
def update_line_dict(betting_games: list[BettingGame]):
  for game in betting_games:
    book_line_to_use = None
    if not game.lines:
      continue
    for book_line in game.lines:
      if book_line.provider == "consensus":
        book_line_to_use = book_line
        break
      elif book_line.provider == "Bovada":
        book_line_to_use = book_line
    if book_line_to_use == None:
      book_line_to_use = game.lines[0]
    lines_dict[game.id] = book_line_to_use


In [ ]:
all_lines = [lines_2020, lines_2021, lines_2022, lines_2023, lines_2024, lines_2025]
for lines_for_year in all_lines:
  update_line_dict(lines_for_year)

In [ ]:
lines_dict

defaultdict(cfbd.models.game_line.GameLine,
            {401237129: GameLine(provider='consensus', spread=-17, formatted_spread='Florida -17', spread_open=None, over_under=62, over_under_open=None, home_moneyline=None, away_moneyline=None),
             401249018: GameLine(provider='consensus', spread=-8, formatted_spread='Air Force -8', spread_open=None, over_under=55.5, over_under_open=None, home_moneyline=None, away_moneyline=None),
             401237135: GameLine(provider='consensus', spread=-31.5, formatted_spread='Alabama -31.5', spread_open=None, over_under=57.5, over_under_open=None, home_moneyline=None, away_moneyline=None),
             401247326: GameLine(provider='consensus', spread=-21, formatted_spread='Ohio State -21', spread_open=None, over_under=67.5, over_under_open=None, home_moneyline=None, away_moneyline=None),
             401237136: GameLine(provider='consensus', spread=1.5, formatted_spread='LSU -1.5', spread_open=None, over_under=65, over_under_open=None, home

**Get Weather 2025**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2025
    season_type = "regular"
    try:
        weather_2025 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2025[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401756846, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2025, 8, 23, 16, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Kansas State', home_conference='Big 12', away_team='Iowa State', away_conference='Big 12', venue_id=3504, venue='Aviva Stadium', temperature=67.6, dew_point=64, humidity=88, precipitation=0, snowfall=0, wind_direction=130, wind_speed=4.3, pressure=1019, weather_condition_code=4, weather_condition='Overcast'),
 GameWeather(id=401756847, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2025, 8, 23, 22, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Kansas', home_conference='Big 12', away_team='Fresno State', away_conference='Mountain West', venue_id=3833, venue='Memorial Stadium', temperature=81.3, dew_point=61, humidity=50, precipitation=0, snowfall=0, wind_direction=356, wind_speed=

**Get Weather 2024**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    weather_api = cfbd.GamesApi(api_client)
    year = 2024
    season_type = "regular"
    try:
        weather_2024 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2024[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401636607, season=2024, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2024, 8, 30, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Utah', home_conference='Big 12', away_team='Southern Utah', away_conference='UAC', venue_id=587, venue='Rice-Eccles Stadium', temperature=76.5, dew_point=29.5, humidity=18, precipitation=0, snowfall=0, wind_direction=329, wind_speed=5.8, pressure=1017.2, weather_condition_code=2, weather_condition='Fair'),
 GameWeather(id=401643775, season=2024, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2024, 8, 31, 21, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='South Alabama', home_conference='Sun Belt', away_team='North Texas', away_conference='American Athletic', venue_id=6526, venue='Hancock Whitney Stadium', temperature=85.6, dew_point=72.5, humidity=65, precipitation=0, snowfall=0, wind_direction=

**Weather 2023**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2023
    season_type = "regular"
    try:
        weather_2023 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2023[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401520362, season=2023, week=10, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2023, 11, 4, 23, 45, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Alabama', home_conference='SEC', away_team='LSU', away_conference='SEC', venue_id=3657, venue='Bryant-Denny Stadium', temperature=60.6, dew_point=43.3, humidity=53, precipitation=0, snowfall=0, wind_direction=49, wind_speed=4.7, pressure=1022, weather_condition_code=1, weather_condition='Clear'),
 GameWeather(id=401531343, season=2023, week=10, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2023, 11, 4, 20, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Tulsa', home_conference='American Athletic', away_team='Charlotte', away_conference='American Athletic', venue_id=3674, venue='Chapman Stadium', temperature=71.6, dew_point=54.5, humidity=55, precipitation=0, snowfall=0, wind_direction=120, wind_speed

**Weather 2022**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2022
    season_type = "regular"
    try:
        weather_2022 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2022[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401426582, season=2022, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2022, 10, 8, 22, 0, tzinfo=datetime.timezone.utc), game_indoors=True, home_team='UTSA', home_conference='Conference USA', away_team='Western Kentucky', away_conference='Conference USA', venue_id=3604, venue='Alamodome', temperature=87.6, dew_point=48.4, humidity=26, precipitation=0, snowfall=0, wind_direction=50, wind_speed=8.1, pressure=1019, weather_condition_code=1, weather_condition='Clear'),
 GameWeather(id=401404008, season=2022, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2022, 10, 9, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Arizona', home_conference='Pac-12', away_team='Oregon', away_conference='Pac-12', venue_id=3619, venue='Arizona Stadium', temperature=72.7, dew_point=49.6, humidity=44, precipitation=0, snowfall=0, wind_direction=110, wind_speed=10.3, 

**Weather 2021**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2021
    season_type = "regular"
    try:
        weather_2021 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2021[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401309833, season=2021, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2021, 8, 28, 19, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='UCLA', home_conference='Pac-12', away_team="Hawai'i", away_conference='Mountain West', venue_id=1056, venue='Rose Bowl', temperature=85.5, dew_point=50.5, humidity=30, precipitation=0, snowfall=0, wind_direction=168, wind_speed=4.9, pressure=1012.2, weather_condition_code=0, weather_condition=None),
 GameWeather(id=401309836, season=2021, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2021, 9, 4, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Colorado', home_conference='Pac-12', away_team='Northern Colorado', away_conference='Big Sky', venue_id=3726, venue='Folsom Field', temperature=66, dew_point=53.4, humidity=64, precipitation=0, snowfall=0, wind_direction=360, wind_speed=8.1, pressure=1

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2020
    season_type = "regular"
    try:
        weather_2020 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2020[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401212522, season=2020, week=3, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2020, 9, 19, 16, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Tulane', home_conference='American Athletic', away_team='Navy', away_conference='American Athletic', venue_id=4729, venue='Yulman Stadium', temperature=73.8, dew_point=70.3, humidity=88.8, precipitation=0.039, snowfall=0, wind_direction=60, wind_speed=21.1, pressure=1015.2, weather_condition_code=7, weather_condition='Light Rain'),
 GameWeather(id=401215308, season=2020, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2020, 10, 10, 19, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Troy', home_conference='Sun Belt', away_team='Texas State', away_conference='Sun Belt', venue_id=3975, venue='Veterans Memorial Stadium (AL)', temperature=77.2, dew_point=74.3, humidity=90.5, precipitation=0.047, sno

**Weather 2023**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2023
    season_type = "regular"
    try:
        weather_2023 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2023[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401520362, season=2023, week=10, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2023, 11, 4, 23, 45, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Alabama', home_conference='SEC', away_team='LSU', away_conference='SEC', venue_id=3657, venue='Bryant-Denny Stadium', temperature=60.6, dew_point=43.3, humidity=53, precipitation=0, snowfall=0, wind_direction=49, wind_speed=4.7, pressure=1022, weather_condition_code=1, weather_condition='Clear'),
 GameWeather(id=401531343, season=2023, week=10, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2023, 11, 4, 20, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Tulsa', home_conference='American Athletic', away_team='Charlotte', away_conference='American Athletic', venue_id=3674, venue='Chapman Stadium', temperature=71.6, dew_point=54.5, humidity=55, precipitation=0, snowfall=0, wind_direction=120, wind_speed

**Weather 2022**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    weather_api = cfbd.GamesApi(api_client)
    year = 2022
    season_type = "regular"
    try:
        weather_2022 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2022[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401426582, season=2022, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2022, 10, 8, 22, 0, tzinfo=datetime.timezone.utc), game_indoors=True, home_team='UTSA', home_conference='Conference USA', away_team='Western Kentucky', away_conference='Conference USA', venue_id=3604, venue='Alamodome', temperature=87.6, dew_point=48.4, humidity=26, precipitation=0, snowfall=0, wind_direction=50, wind_speed=8.1, pressure=1019, weather_condition_code=1, weather_condition='Clear'),
 GameWeather(id=401404008, season=2022, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2022, 10, 9, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Arizona', home_conference='Pac-12', away_team='Oregon', away_conference='Pac-12', venue_id=3619, venue='Arizona Stadium', temperature=72.7, dew_point=49.6, humidity=44, precipitation=0, snowfall=0, wind_direction=110, wind_speed=10.3, 

**Weather 2021**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2021
    season_type = "regular"
    try:
        weather_2021 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2021[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401309833, season=2021, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2021, 8, 28, 19, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='UCLA', home_conference='Pac-12', away_team="Hawai'i", away_conference='Mountain West', venue_id=1056, venue='Rose Bowl', temperature=85.5, dew_point=50.5, humidity=30, precipitation=0, snowfall=0, wind_direction=168, wind_speed=4.9, pressure=1012.2, weather_condition_code=0, weather_condition=None),
 GameWeather(id=401309836, season=2021, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2021, 9, 4, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Colorado', home_conference='Pac-12', away_team='Northern Colorado', away_conference='Big Sky', venue_id=3726, venue='Folsom Field', temperature=66, dew_point=53.4, humidity=64, precipitation=0, snowfall=0, wind_direction=360, wind_speed=8.1, pressure=1

**Weather 2020**

In [ ]:
with cfbd.ApiClient(configuration) as api_client:
    weather_api = cfbd.GamesApi(api_client)
    year = 2020
    season_type = "regular"
    try:
        weather_2020 = weather_api.get_weather(year=year, season_type=season_type, classification="fbs")
        print("The response of GamesApi->get_weather:\n")
        pprint(weather_2020[:5])
    except Exception as e:
        print("Exception when calling GamesApi->get_weather: %s\n" % e)

The response of GamesApi->get_weather:

[GameWeather(id=401212522, season=2020, week=3, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2020, 9, 19, 16, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Tulane', home_conference='American Athletic', away_team='Navy', away_conference='American Athletic', venue_id=4729, venue='Yulman Stadium', temperature=73.8, dew_point=70.3, humidity=88.8, precipitation=0.039, snowfall=0, wind_direction=60, wind_speed=21.1, pressure=1015.2, weather_condition_code=7, weather_condition='Light Rain'),
 GameWeather(id=401215308, season=2020, week=6, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2020, 10, 10, 19, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Troy', home_conference='Sun Belt', away_team='Texas State', away_conference='Sun Belt', venue_id=3975, venue='Veterans Memorial Stadium (AL)', temperature=77.2, dew_point=74.3, humidity=90.5, precipitation=0.047, sno

In [ ]:
def update_weather_map(weather_observations: list[GameWeather]):
  for observation in weather_observations:
    weather_dict[observation.id] = observation

In [ ]:
weather_dict = defaultdict(GameWeather)
weather_years = [weather_2025, weather_2024, weather_2023, weather_2022, weather_2021, weather_2020]
for year in weather_years:
  update_weather_map(year)

In [ ]:
weather_dict

defaultdict(cfbd.models.game_weather.GameWeather,
            {401756846: GameWeather(id=401756846, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2025, 8, 23, 16, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Kansas State', home_conference='Big 12', away_team='Iowa State', away_conference='Big 12', venue_id=3504, venue='Aviva Stadium', temperature=67.6, dew_point=64, humidity=88, precipitation=0, snowfall=0, wind_direction=130, wind_speed=4.3, pressure=1019, weather_condition_code=4, weather_condition='Overcast'),
             401756847: GameWeather(id=401756847, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2025, 8, 23, 22, 30, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Kansas', home_conference='Big 12', away_team='Fresno State', away_conference='Mountain West', venue_id=3833, venue='Memorial Stadium', temperature=81.3, dew_point=61, humidity=50, prec

# Putting it all together

The big Kahuna

In [633]:
def update_game_records(games: list[Game]):

  def assign_coach(coaches_to_consider):
    if coaches_to_consider == []:
      return [CoachLite()]
    fired = coaches_to_consider[0]
    interim = coaches_to_consider[1]
    if week > fired.last_week_coached:
      return [interim]
    else:
      return [fired]

  def covered():
    if year == 2025 and week == CURR_WEEK:
      return None
    try:
      point_delta = game.away_points - game.home_points
      if spread < 0:
        return point_delta < spread
      else:
        return point_delta > spread
    except:
      print(f"Exception occurred on {game.id} where week is {game.week} and season is {year}")

  for game in games:
    curr = GameRecord()
    curr.id = game.id
    # Fields that will be used throughout construction
    home = game.home_id
    away = game.away_id
    game_id = game.id
    year = game.season
    week = game.week
    if game_id not in lines_dict or lines_dict[game_id].spread == None:
      continue
    # print(home, away, game_id)
    # Fields from the Game response
    curr.season = game.season
    curr.week = week
    curr.start_date = game.start_date
    curr.neutral = int(game.neutral_site)
    curr.conference_game = int(game.conference_game)
    curr.venue = game.venue_id
    curr.home_team = home
    curr.home_conference = game.home_conference
    curr.home_classification = game.home_classification
    curr.home_pregame_elo = game.home_pregame_elo
    curr.away_team = away
    curr.away_conference = game.away_conference
    curr.away_pregame_elo = game.away_pregame_elo

    # Fields from the coaches dict
    home_coach = coaches_dict[(home, year)]
    if len(home_coach) != 1:
      home_coach = assign_coach(home_coach)
    away_coach = coaches_dict[(away, year)]
    if len(away_coach) != 1:
      away_coach = assign_coach(away_coach)
    home_coach = home_coach[0]
    away_coach = away_coach[0]
    curr.home_coach_name = home_coach.name
    curr.home_coach_tenure = home_coach.tenure
    curr.home_coach_interim = int(home_coach.interim)
    curr.away_coach_name = away_coach.name
    curr.away_coach_tenure = away_coach.tenure
    curr.away_coach_interim = int(away_coach.interim)

    # Fields from the lines dict
    line = lines_dict[game_id]
    spread = line.spread
    curr.spread = spread
    curr.spread_open = line.spread_open
    curr.over_under = line.over_under
    curr.over_under_open = line.over_under_open
    curr.home_moneyline = line.home_moneyline
    curr.away_moneyline = line.away_moneyline

    # Fields from the weather dict
    if game_id in weather_dict:
      game_weather = weather_dict[game_id]
      curr.game_indoors = int(game_weather.game_indoors)
      curr.temperature = game_weather.temperature
      curr.dew_point = game_weather.dew_point
      curr.humidity = game_weather.humidity
      curr.precipitation = game_weather.precipitation
      curr.snowfall = game_weather.snowfall
      curr.wind_dir = game_weather.wind_direction
      curr.wind_speed = game_weather.wind_speed
      curr.atm_pressure = game_weather.pressure
      curr.weather_condition = game_weather.weather_condition_code

    covered_spread = covered()
    curr.covered = covered_spread


    game_records_dataset.append(curr)

In [634]:
game_records_dataset = []
update_game_records(games_2025_subset)

In [635]:
game_records_dataset = []
seasons = [games_2025_subset, games_2024, games_2023, games_2022, games_2021, games_2020]
for season in seasons:
  update_game_records(season)

In [639]:
game_records_dataset[6]

GameRecord(id=401767410, season=2025, week=1, start_date=datetime.datetime(2025, 8, 23, 23, 30, tzinfo=datetime.timezone.utc), neutral=1, conference_game=0, venue=4246, home_team=2582, home_conference='SWAC', home_classification=<DivisionClassification.FCS: 'fcs'>, home_pregame_elo=None, away_team=2428, away_conference='MEAC', away_classification='', away_pregame_elo=None, home_coach_name='', home_coach_tenure=0, home_coach_interim=0, away_coach_name='', away_coach_tenure=0, away_coach_interim=0, spread=10.5, spread_open=None, over_under=47.5, over_under_open=None, home_moneyline=None, away_moneyline=None, game_indoors=0, temperature=inf, dew_point=inf, humidity=-1, precipitation=-1, snowfall=-1, wind_dir=inf, wind_speed=-1, atm_pressure=inf, weather_condition=-1, covered=True)

In [645]:
no_temp = [game for game in game_records_dataset if game.temperature == float("inf")]
len(no_temp)

1882

In [640]:
[game for game in games_2025 if game.id == 401767410]

[Game(id=401767410, season=2025, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2025, 8, 23, 23, 30, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=False, attendance=None, venue_id=4246, venue='Center Parc Stadium', home_id=2582, home_team='Southern', home_conference='SWAC', home_classification=<DivisionClassification.FCS: 'fcs'>, home_points=14, home_line_scores=[7, 7, 0, 0], home_postgame_win_probability=0.11636106669902802, home_pregame_elo=None, home_postgame_elo=None, away_id=2428, away_team='North Carolina Central', away_conference='MEAC', away_classification=<DivisionClassification.FCS: 'fcs'>, away_points=31, away_line_scores=[7, 10, 14, 0], away_postgame_win_probability=0.883638933300972, away_pregame_elo=None, away_postgame_elo=None, excitement_index=3.815677850005746, highlights='', notes='Cricket MEAC/SWAC Challenge Kickoff')]

In [637]:
df = pd.DataFrame(game_records_dataset)
df.to_csv("2025_pre_week4_v2.csv", index=False)

In [638]:
from google.colab import files
files.download("2025_pre_week4_v2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Build Game Records Dataset

# Optional: Download Games

Download the games that were just requested

In [ ]:
df = pd.DataFrame([vars(g) for g in games_response])
# Uncomment if you want a csv
# df.to_csv("games.csv", index=False)

Download the file generated, will update this to a direct drive integration later

In [ ]:
# from google.colab import files
# files.download("games.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Research - Weather Info
Weather info includes historical data as well as predictions

In [ ]:
api_response[]

GameWeather(id=401636607, season=2024, week=1, season_type=<SeasonType.REGULAR: 'regular'>, start_time=datetime.datetime(2024, 8, 30, 1, 0, tzinfo=datetime.timezone.utc), game_indoors=False, home_team='Utah', home_conference='Big 12', away_team='Southern Utah', away_conference='UAC', venue_id=587, venue='Rice-Eccles Stadium', temperature=76.5, dew_point=29.5, humidity=18, precipitation=0, snowfall=0, wind_direction=329, wind_speed=5.8, pressure=1017.2, weather_condition_code=2, weather_condition='Fair')

# Research - Lines

The different lines by year are as follows:

2020 -
{'consensus', 'Caesars', 'numberfire', 'SugarHouse', 'Caesars (Pennsylvania)', 'teamrankings', 'Bovada', 'William Hill (New Jersey)'}

2021 -
{'consensus', 'Caesars Sportsbook (Colorado)', 'numberfire', 'Caesars (Pennsylvania)', 'teamrankings', 'William Hill (New Jersey)', 'Bovada'}

2022 -
{'consensus', 'Caesars Sportsbook (Colorado)', 'teamrankings', 'William Hill (New Jersey)', 'Bovada'}

2023 -
{'consensus', 'Caesars Sportsbook (Colorado)', 'William Hill (New Jersey)', 'Bovada', 'teamrankings', 'ESPN Bet', 'DraftKings'}

2024 -
{'DraftKings', 'Bovada', 'ESPN Bet'}

Consistent across years:
{'Bovada'}

**Verdict**
The line priority will be 1) consensus, 2) Bovada, 3) whatever comes first

In [ ]:
distinct_books_20 = set()

# for line_wrapper in lines:
#   for line in line_wrapper.lines:
#     distinct_books.add(line.provider)

for line_wrapper in lines_2020:
  for line in line_wrapper.lines:
    distinct_books_20.add(line.provider)

print("2020")
print(distinct_books_20)

distinct_books_21 = set()
for line_wrapper in lines_2021:
  for line in line_wrapper.lines:
    distinct_books_21.add(line.provider)

print("2021")
print(distinct_books_21)

distinct_books_22 = set()
for line_wrapper in lines_2022:
  for line in line_wrapper.lines:
    distinct_books_22.add(line.provider)

print("2022")
print(distinct_books_22)

distinct_books_23 = set()
for line_wrapper in lines_2023:
  for line in line_wrapper.lines:
    distinct_books_23.add(line.provider)

print("2023")
print(distinct_books_23)

distinct_books_24 = set()
for line_wrapper in lines:
  for line in line_wrapper.lines:
    distinct_books_24.add(line.provider)

print("2024")
print(distinct_books_24)

print("intersection")
shared = set.intersection(distinct_books_24, distinct_books_23, distinct_books_22, distinct_books_21, distinct_books_20)
print(shared)

2020
{'consensus', 'Caesars', 'numberfire', 'SugarHouse', 'Caesars (Pennsylvania)', 'teamrankings', 'Bovada', 'William Hill (New Jersey)'}
2021
{'consensus', 'Caesars Sportsbook (Colorado)', 'numberfire', 'Caesars (Pennsylvania)', 'teamrankings', 'William Hill (New Jersey)', 'Bovada'}
2022
{'consensus', 'Caesars Sportsbook (Colorado)', 'teamrankings', 'William Hill (New Jersey)', 'Bovada'}
2023
{'consensus', 'Caesars Sportsbook (Colorado)', 'William Hill (New Jersey)', 'Bovada', 'teamrankings', 'ESPN Bet', 'DraftKings'}
2024
{'DraftKings', 'Bovada', 'ESPN Bet'}
intersection
{'Bovada'}


In [ ]:
[line for line in lines_2024 if line.id == 401654609]

[BettingGame(id=401654609, season=2024, season_type=<SeasonType.REGULAR: 'regular'>, week=10, start_date=datetime.datetime(2024, 11, 2, 16, 0, tzinfo=datetime.timezone.utc), home_team_id=2523, home_team='Robert Morris', home_conference='NEC', home_classification=<DivisionClassification.FCS: 'fcs'>, home_score=6, away_team_id=2771, away_team='Merrimack', away_conference='FCS Independents', away_classification=<DivisionClassification.FCS: 'fcs'>, away_score=0, lines=[GameLine(provider='ESPN Bet', spread=1.5, formatted_spread='Merrimack -1.5', spread_open=None, over_under=48.5, over_under_open=None, home_moneyline=None, away_moneyline=None)])]

# Research - Coaches

Look at coaches and their respective teams that were fired, and figure out how to deal with them.

**Verdict** Assume that there will be at most one interim coach. Mark the last game coached by the first coach in a given season by looking at the number of games they coached in that season and matching that against the weeks in which the team played.

**Get Coaches**

In [ ]:
missing_hire_dates = [coach for coach in coaches if coach.hire_date == None]
missing_hire_dates[0].to_dict()
# num_coaches = len(coaches)
# print(f"missing hired dates: {len(missing_hire_dates)} for {num_coaches} total coaches")

{'firstName': 'Pete',
 'lastName': 'Alamar',
 'seasons': [{'school': 'Rice',
   'year': 2024,
   'games': 4,
   'wins': 2,
   'losses': 2,
   'ties': 0,
   'srs': -8.6,
   'spOverall': -11.4,
   'spOffense': 20.5,
   'spDefense': 30.4,
   'preseasonRank': None,
   'postseasonRank': None}],
 'hireDate': None}

In [ ]:
coaches_per_school = defaultdict(int)
schools_with_firings = set()
for coach in coaches:
  school = coach.seasons[0].school
  coaches_per_school[school] += 1
  if coaches_per_school[school] > 1:
    schools_with_firings.add(school)

print(schools_with_firings)

{'Massachusetts', 'East Carolina', 'West Virginia', 'Georgia State', 'Tulsa', 'Utah State', 'Charlotte', 'Temple', 'Southern Miss', 'Florida Atlantic', 'Houston', 'Washington State', 'Rice', 'Ball State', 'Kennesaw State', 'Sam Houston', 'North Carolina', 'Buffalo'}


In [ ]:

fired_coaches = [coach for coach in coaches if coach.seasons[0].school in schools_with_firings and coach.hire_date != None]
interim_coaches = [coach for coach in coaches if coach.hire_date == None]
for coach in fired_coaches:
  print(coach.to_dict())
print()
print("Interim Coaches")
for coach in interim_coaches:
  print(coach.to_dict())

{'firstName': 'Blake', 'lastName': 'Anderson', 'hireDate': datetime.datetime(2020, 12, 12, 0, 0, tzinfo=datetime.timezone.utc), 'seasons': [{'school': 'Utah State', 'year': 2024, 'games': 0, 'wins': 0, 'losses': 0, 'ties': 0, 'srs': -11.7, 'spOverall': -10.4, 'spOffense': 30.2, 'spDefense': 40.3, 'preseasonRank': None, 'postseasonRank': None}]}
{'firstName': 'Mike', 'lastName': 'Bloomgren', 'hireDate': datetime.datetime(2017, 12, 6, 0, 0, tzinfo=datetime.timezone.utc), 'seasons': [{'school': 'Rice', 'year': 2024, 'games': 8, 'wins': 2, 'losses': 6, 'ties': 0, 'srs': -8.6, 'spOverall': -11.4, 'spOffense': 20.5, 'spDefense': 30.4, 'preseasonRank': None, 'postseasonRank': None}]}
{'firstName': 'Don', 'lastName': 'Brown', 'hireDate': datetime.datetime(2021, 11, 22, 0, 0, tzinfo=datetime.timezone.utc), 'seasons': [{'school': 'Massachusetts', 'year': 2024, 'games': 10, 'wins': 2, 'losses': 8, 'ties': 0, 'srs': -17.4, 'spOverall': -21.9, 'spOffense': 20.5, 'spDefense': 40.3, 'preseasonRank': 

In [ ]:
a = CoachLite(name="Mike Locksley", tenure = 4, fired=False, interim=False, as_of= 2024)
a.to_dict()

{'name': 'Mike Locksley',
 'tenure': 4,
 'fired': False,
 'interim': False,
 'last_week_coached': None,
 'as_of': 2024}

In [ ]:
[coach for coach in coaches if coach.seasons[0].school == "Rice"]

[Coach(first_name='Pete', last_name='Alamar', hire_date=None, seasons=[CoachSeason(school='Rice', year=2024, games=4, wins=2, losses=2, ties=0, preseason_rank=None, postseason_rank=None, srs=-8.6, sp_overall=-11.4, sp_offense=20.5, sp_defense=30.4)]),
 Coach(first_name='Mike', last_name='Bloomgren', hire_date=datetime.datetime(2017, 12, 6, 0, 0, tzinfo=datetime.timezone.utc), seasons=[CoachSeason(school='Rice', year=2024, games=8, wins=2, losses=6, ties=0, preseason_rank=None, postseason_rank=None, srs=-8.6, sp_overall=-11.4, sp_offense=20.5, sp_defense=30.4)])]

In [ ]:
rice_games = [game.week for game in games_response if game.home_team == "Rice" or game.away_team == "Rice"]
sorted(rice_games)
# sorted_rice = sorted(rice_games, key=lambda x: x.week)
# for game in sorted_rice:
#   print(game.week)

[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14]

In [ ]:
rice = [coach for coach in coaches if coach.seasons[0].school == "Rice"]
update_coaches_dict(rice, 2024, games_response)

CoachLite(name='Pete Alamar', tenure=0, fired=False, interim=True, last_week_coached=None, season=2024, season_games_coached=4) CoachLite(name='Mike Bloomgren', tenure=6, fired=True, interim=False, last_week_coached=9, season=2024, season_games_coached=8)


In [ ]:
coaches_dict[("East Carolina", 2024)]

[CoachLite(name='Mike Houston', tenure=5, fired=True, interim=False, last_week_coached=8, season=2024, season_games_coached=7),
 CoachLite(name='Blake Harrell', tenure=0, fired=False, interim=True, last_week_coached=None, season=2024, season_games_coached=6)]

In [ ]:
utah_st = coaches_2021[3:5]
one = utah_st[0]
two = utah_st[1]
one.first_name == two.first_name and one.last_name == two.last_name

True

# Research - General Testing

In [ ]:
g = GameRecord(season = 2024)
# g.week = 1
# g.startDate = datetime.now(tz=timezone.utc)
# g.neutral = False
# g.conference_game = False
# g.venue = "Blah"
# g.home_team = "LSU"
# g.home_conference = "SEC"
g.to_dict()

{'season': 2024,
 'week': None,
 'startDate': None,
 'neutral': None,
 'conference_game': None,
 'venue': '',
 'home_team': '',
 'home_conference': '',
 'home_classification': '',
 'home_pregame_elo': -1,
 'away_team': '',
 'away_conference': '',
 'away_classification': '',
 'away_pregame_elo': -1,
 'home_coach_name': '',
 'home_coach_tenure': -1,
 'home_coach_interim': None,
 'away_coach_name': '',
 'away_coach_tenure': -1,
 'away_coach_interim': None,
 'spread': inf,
 'spread_open': inf,
 'over_under': -1,
 'over_under_open': -1,
 'home_moneyline': 0,
 'away_moneyline': 0,
 'game_indoors': None,
 'temperature': inf,
 'dew_point': inf,
 'humidity': -1,
 'precipitation': -1,
 'snowfall': -1,
 'wind_dir': inf,
 'wind_speed': -1,
 'atm_pressure': inf,
 'weather_condition': '',
 'result': None}

In [ ]:
games_records_dataset = []

In [ ]:
# Big 10 games
bt_games = [game for game in games_response if game.away_conference == "Big Ten" and game.home_conference == "Big Ten"]
bt_games[30]

Game(id=401628522, season=2024, week=8, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2024, 10, 19, 16, 0, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=True, conference_game=True, attendance=12023, venue_id=5960, venue='Lanny and Sharon Martin Stadium', home_id=77, home_team='Northwestern', home_conference='Big Ten', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=3, home_line_scores=[0, 0, 3, 0], home_postgame_win_probability=0.0059807059089865165, home_pregame_elo=1569, home_postgame_elo=1543, away_id=275, away_team='Wisconsin', away_conference='Big Ten', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=23, away_line_scores=[0, 14, 7, 2], away_postgame_win_probability=0.9940192940910135, away_pregame_elo=1729, away_postgame_elo=1755, excitement_index=3.6478457494, highlights='', notes=None)

In [ ]:
rice_games = [game for game in games_response if game.home_team == "Rice" or game.away_team == "Rice"]
build_game_records(games_response)

In [ ]:
coaches_dict

defaultdict(list,
            {('Rice',
              2024): [CoachLite(name='Mike Bloomgren', tenure=6, fired=True, interim=False, last_week_coached=9, season=2024, season_games_coached=8), CoachLite(name='Pete Alamar', tenure=0, fired=False, interim=True, last_week_coached=None, season=2024, season_games_coached=4)],
             ('Ohio',
              2024): [CoachLite(name='Tim Albin', tenure=3, fired=False, interim=False, last_week_coached=None, season=2024, season_games_coached=14)],
             ('Utah State',
              2024): [CoachLite(name='Blake Anderson', tenure=3, fired=True, interim=False, last_week_coached=14, season=2024, season_games_coached=0), CoachLite(name='Nate Dreiling', tenure=0, fired=False, interim=True, last_week_coached=None, season=2024, season_games_coached=12)],
             ('South Alabama',
              2024): [CoachLite(name='Major Applewhite', tenure=0, fired=False, interim=False, last_week_coached=None, season=2024, season_games_coached=13)],
  

In [ ]:
game_records_dataset[0]

GameRecord(id=401635525, season=2024, week=1, start_date=datetime.datetime(2024, 8, 24, 16, 0, tzinfo=datetime.timezone.utc), neutral=True, conference_game=True, venue='Aviva Stadium', home_team='Georgia Tech', home_conference='ACC', home_classification=<DivisionClassification.FBS: 'fbs'>, home_pregame_elo=1497, away_team='Florida State', away_conference='ACC', away_classification='', away_pregame_elo=1692, home_coach_name='Brent Key', home_coach_tenure=1, home_coach_interim=False, away_coach_name='Mike Norvell', away_coach_tenure=4, away_coach_interim=False, spread=10, spread_open=12, over_under=54.5, over_under_open=57, home_moneyline=345, away_moneyline=-470, game_indoors=None, temperature=inf, dew_point=inf, humidity=-1, precipitation=-1, snowfall=-1, wind_dir=inf, wind_speed=-1, atm_pressure=inf, weather_condition='', result=None)

**Notes**


*   It seems like it will be easier to use the built in package. Can use the pregame ELO metric as a team strength metric
*   Will probably want to filter such that the home team is always in FBS
*   Coaches only returns an array for seasons if you don't request a specific season
* The to_str() method changes from snake case to camel case


**Conclusion**
No need to use the venues API, because the details are already included in the team API. Is anything in the "team" API actually valuable?